<!-- docusaurus_head_meta::start
---
title: Introduction to Evaluations
---
docusaurus_head_meta::end -->


# Introduction to Evaluations

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

Weave is a toolkit for developing AI-powered applications.

This notebook demonstrates how to evaluate a model or function using Weave’s Evaluation API. Evaluation is a core concept in Weave that helps you measure and iterate on your application by running it against a dataset of examples and scoring the outputs using custom-defined functions. You'll define a simple model, create a labeled dataset, track scoring functions with `@weave.op`, and run an evaluation that automatically tracks results in the Weave UI. This forms the foundation for more advanced workflows like LLM fine-tuning, regression testing, or model comparison.

To get started, complete the prerequisites. Then, define a Weave `Model` with a `predict` method, create a labeled dataset and scoring function, and run an evaluation using `weave.Evaluation.evaluate()`.

## 🐝 Running your first evaluation

In this example, we're using W&B Inference. [Learn more](https://docs.wandb.ai/inference) about our inference API.
Using another provider? [We support all major clients and frameworks](https://docs.wandb.ai/weave/guides/integrations).

We also have an OpenAI version of this example below.

In [ ]:
# Ensure your dependencies are installed with:
!pip install openai pandas weave

# Find your wandb API key at: https://wandb.ai/authorize
# Ensure that your wandb API key is available at:
# os.environ['WANDB_API_KEY'] = "<your_wandb_api_key>"

import asyncio
import os
import re
from textwrap import dedent

import weave
from openai import OpenAI


class JsonModel(weave.Model):
    prompt: weave.Prompt = weave.StringPrompt(
        dedent("""
You are an assistant that answers questions about JSON data provided by the user. The JSON data represents structured information of various kinds, and may be deeply nested. In the first user message, you will receive the JSON data under a label called 'context', and a question under a label called 'question'. Your job is to answer the question with as much accuracy and brevity as possible. Give only the answer with no preamble. You must output the answer in XML format, between <answer> and </answer> tags.
""")
    )
    model: str = "OpenPipe/Qwen3-14B-Instruct"
    _client: OpenAI

    def __init__(self):
        super().__init__()
        self._client = OpenAI(
            base_url='https://api.inference.wandb.ai/v1',
            api_key=os.environ['WANDB_API_KEY'],
            project='martin-test/another-new-test',
        )

    @weave.op
    def predict(self, context: str, question: str) -> str:
        response = self._client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.prompt.format()},
                {
                    "role": "user",
                    "content": f"Context: {context}\nQuestion: {question}",
                },
            ],
        )
        return response.choices[0].message.content


@weave.op
def correct_answer_format(answer: str, output: str) -> dict[str, bool]:
    parsed_output = re.search(r"<answer>(.*?)</answer>", output, re.DOTALL)
    if parsed_output is None:
        return {"correct_answer": False, "correct_format": False}
    return {"correct_answer": parsed_output.group(1) == answer, "correct_format": True}


if __name__ == "__main__":
    if not os.environ.get('WANDB_API_KEY'):
        print("WANDB_API_KEY is not set - make sure to export it in your environment or assign it in this script")
        exit(1)

    # Find your wandb API key at: https://wandb.ai/authorize
    weave.init("martin-test/another-new-test")

    jsonqa = weave.Dataset.from_uri(
        "weave:///wandb/json-qa/object/json-qa:v3"
    ).to_pandas()

    model = JsonModel()

    eval = weave.Evaluation(
        name="json-qa-eval",
        dataset=weave.Dataset.from_pandas(jsonqa),
        scorers=[correct_answer_format],
    )

    asyncio.run(eval.evaluate(model))

## 🐝 Running your first evaluation (using OpenAI)

Your can find your OpenAI API keys here: https://platform.openai.com/api-keys

In [ ]:
# Ensure your dependencies are installed with:
!pip install openai pandas weave

# Find your OpenAI API key at: https://platform.openai.com/api-keys
# Ensure that your OpenAI API key is available at:
# os.environ['OPENAI_API_KEY'] = "<your_openai_api_key>"

import asyncio
import os
import re
from textwrap import dedent

import openai
import weave


class JsonModel(weave.Model):
    prompt: weave.Prompt = weave.StringPrompt(
        dedent("""
You are an assistant that answers questions about JSON data provided by the user. The JSON data represents structured information of various kinds, and may be deeply nested. In the first user message, you will receive the JSON data under a label called 'context', and a question under a label called 'question'. Your job is to answer the question with as much accuracy and brevity as possible. Give only the answer with no preamble. You must output the answer in XML format, between <answer> and </answer> tags.
""")
    )
    model: str = "gpt-4.1-nano"
    _client: openai.OpenAI

    def __init__(self):
        super().__init__()
        self._client = openai.OpenAI()

    @weave.op
    def predict(self, context: str, question: str) -> str:
        response = self._client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.prompt.format()},
                {
                    "role": "user",
                    "content": f"Context: {context}\nQuestion: {question}",
                },
            ],
        )
        assert response.choices[0].message.content is not None
        return response.choices[0].message.content


@weave.op
def correct_answer_format(answer: str, output: str) -> dict[str, bool]:
    parsed_output = re.search(r"<answer>(.*?)</answer>", output, re.DOTALL)
    if parsed_output is None:
        return {"correct_answer": False, "correct_format": False}
    return {"correct_answer": parsed_output.group(1) == answer, "correct_format": True}


if __name__ == "__main__":
    if not os.environ.get('OPENAI_API_KEY'):
        print("OPENAI_API_KEY is not set - make sure to export it in your environment or assign it in this script")
        exit(1)

    # Find your wandb API key at: https://wandb.ai/authorize
    weave.init("martin-test/another-new-test")

    jsonqa = weave.Dataset.from_uri(
        "weave:///wandb/json-qa/object/json-qa:v3"
    ).to_pandas()

    model = JsonModel()

    eval = weave.Evaluation(
        name="json-qa-eval",
        dataset=weave.Dataset.from_pandas(jsonqa),
        scorers=[correct_answer_format],
    )

    asyncio.run(eval.evaluate(model))

## 🚀 Looking for more examples?

- Learn how to build an [evlauation pipeline end-to-end](https://weave-docs.wandb.ai/tutorial-eval). 
- Learn how to evaluate a [RAG application by building](https://weave-docs.wandb.ai/tutorial-rag).